In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

In [2]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener

In [3]:
import nltk
import datetime
import json
import time

In [4]:
ckey = "sJBrROFCkgP0mBNTZiixLgoGs"
csecret = "N4NeIgH4bdMYs37kM2gLx7N2eIPnPSTZMXMBYyNmfUlR8GfXG7"
atoken = "881521281979400193-xUJPrzZnBMlnZujwAEHBdpvUcoNxxZs"
asecret = "fAl79BZl1HdihzPBt6VjnS1SDoOm5C8rRUEFPkXfRLL09"

In [5]:
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

In [6]:
api = tweepy.API(auth)

In [7]:
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, tweet_limit=2000):
        self.count = 0
        self.limit = tweet_limit
        self.saveFile = open('rawtweets.json', 'w')
        super(MyStreamListener, self).__init__()

    def on_data(self, data):
        if self.count < self.limit:
            self.saveFile.write(data)
            self.count+=1
            return True
        else:
            self.saveFile.close()
            return False

In [8]:
def get_raw():
    data = []
    with open('rawtweets.json') as f:
        for line in f:
            if line.rstrip():
                data.append(json.loads(line))
    return data

In [9]:
def trim_tweets(data):
    outtweets = [[time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(rawtweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')),
                  rawtweet['created_at'][0:3],
                  rawtweet["user"]["screen_name"], 
                  rawtweet["text"],
                  rawtweet["source"].split("\u003e")[1].split("</a\u003c\/a\u003e")[0][:-3], 
                  rawtweet['user']['location'],
                  rawtweet["retweet_count"],
                  rawtweet['favorite_count'],
                  rawtweet['reply_count'],
                  rawtweet['user']['followers_count'],
                  rawtweet['user']['friends_count'],
                  rawtweet['user']['favourites_count']] for rawtweet in data]
    return outtweets
    print (len(alltweets))

In [10]:
def getDF(trimmed):
    columns = ['Created At', 'Day', 'Screen Name', 'Text', 'Source', 'User Location', 'Retweet Count', 'Favourite Count', 'Reply Count', 'User Followers', 'User Friends', 'User Favorites']
    twDF = pd.DataFrame(trimmed, columns=columns)
    return twDF

In [11]:
def track_topics(topics, number):
    myStream = tweepy.Stream(auth=api.auth, listener=MyStreamListener(tweet_limit=number))
    myStream.filter(track=topics, encoding='utf-8')
    data = get_raw()
    trimmed = trim_tweets(data)
    twDF = getDF(trimmed)
    twDF['Created At'] = pd.to_datetime(twDF['Created At'])
    twDF['Date Only'] = twDF['Created At'].dt.date
    return twDF

In [12]:
topics = ['Tesla', 'Microsoft']

In [13]:
twDF = track_topics(topics, 10)

In [14]:
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [15]:
def get_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    return(snt)

In [16]:
def get_sentDF(tweet_dataframe):
    vadercols = ['neg', 'neu', 'pos', 'compound']
    vaderDF = pd.DataFrame(columns = vadercols)
    tweets = tweet_dataframe["Text"]
    for tweet in range(len(tweets)):
        new_dict = get_sentiment_scores(str(tweets[tweet]))
        newDF = pd.DataFrame(new_dict, index = [tweet])
        vaderDF = pd.concat([vaderDF,newDF])
    twvaderDF = pd.concat([tweet_dataframe,vaderDF], axis=1)
    return twvaderDF

In [17]:
twvaderDF = get_sentDF(twDF)

In [18]:
twvaderDF

,Created At,Day,Screen Name,Text,Source,User Location,Retweet Count,Favourite Count,Reply Count,User Followers,User Friends,User Favorites,Date Only,compound,neg,neu,pos
0,2018-02-21 01:18:42,Wed,unbiasednreal,@Audi @BMWUSA @GEICO @McDonalds @HiltonHotels ...,Twitter for iPhone,None,0,0,0,1,1,0,2018-02-21,0.0000,0.000,1.000,0.000
1,2018-02-21 01:18:45,Wed,Sanjanakhanna19,RT @BradSmi: The people that come to the US to...,Twitter for iPhone,None,0,0,0,1,11,171,2018-02-21,0.7506,0.000,0.748,0.252
2,2018-02-21 01:18:46,Wed,ZephyrArcturus,imaginez dans plusieurs centaines d'années qua...,TweetDeck,destiny - ancient spaceship,0,0,0,234,559,10427,2018-02-21,-0.1280,0.073,0.927,0.000
3,2018-02-21 01:18:47,Wed,Adam_Ak,RT @Shubzy_S: Bazinga! https://t.co/xOQ0Kz6NGX,Twitter for iPhone,None,0,0,0,602,328,624,2018-02-21,0.0000,0.000,1.000,0.000
4,2018-02-21 01:18:49,Wed,HairBandRadio,#80sHairBands #NP: Tesla - Love Song LISTEN NO...,Share.Radionomy.com,"Portland, OR",0,0,0,1208,142,3488,2018-02-21,0.6369,0.000,0.625,0.375
5,2018-02-21 01:18:50,Wed,ImRaunchePubis,#TrumpTrain #2A #elonmusk #tesla #WorldRecord ...,RauncePubis Digital Power,None,0,0,0,14,0,463,2018-02-21,0.0000,0.000,1.000,0.000
6,2018-02-21 01:18:52,Wed,p14nz,RT @hackerstorm: Cryptojacking scripts in Micr...,Twitter for iPhone,Bogor,0,0,0,35,66,9,2018-02-21,0.0000,0.000,1.000,0.000
7,2018-02-21 01:18:53,Wed,aneeshack4,"So now with microsoft, while it’s usually posi...",Twitter for iPhone,"Seattle, WA",0,0,0,594,768,15800,2018-02-21,-0.2960,0.220,0.644,0.136
8,2018-02-21 01:18:54,Wed,matthewshirts,Tesla vehicles now dominates luxury segment in...,Twitter Web Client,Brazil,0,0,0,7819,6592,32711,2018-02-21,0.1531,0.000,0.822,0.178
9,2018-02-21 01:18:53,Wed,startupcrunch,#startup Startup entrepreneurs get history les...,dlvr.it,"Palo Alto, CA",0,0,0,31312,3617,9,2018-02-21,0.0000,0.000,1.000,0.000


In [19]:
sentiment_scores = twvaderDF.groupby('Date Only')['compound'].apply(sum)

In [20]:
sent_scores = pd.DataFrame(sentiment_scores)

In [22]:
sent_scores

,compound
Date Only,
2018-02-21,1.1166
